# Introduction to Semantic Kernel

## Theory Module

### Prerequisites

- Python 3.8 or later
- Basic understanding of Python programming
- OpenAI API key or Azure OpenAI access
- Semantic Kernel library installed

In [1]:
# Setup - call this once at the beginning of the notebook to add the parent directory to the path

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### 1. What is Semantic Kernel?

Semantic Kernel is an open-source SDK that integrates LLMs into applications. Think of it as a bridge between your application and AI models, providing a structured way to:

- Create AI-powered functions
- Combine AI capabilities with traditional programming
- Manage context and memory
- Orchestrate complex AI workflows

Compared to langchain or llama-index Semantic Kernel focuses deeply on workflows.

### Setting up the Kernel

In Semantic Kernel all interactions with a LLM happens through a kernel.
Let's connect to it!

In [2]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.contents import ChatHistory

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

settings = chat_service.get_prompt_execution_settings_class()
settings.temperature = 0.7
settings.top_p = 0.8

kernel.add_service(chat_service)

# Test the kernel
print("Kernel initialized successfully!")

Kernel initialized successfully!


In [4]:
system_message = """
You are Fred. A white duck. He uses emojis all the time,
as you would expect from a good white duck.
"""

chat_function = kernel.add_function(
    prompt=system_message + """{{$chat_history}}{{$user_input}}""",
    function_name="chat",
    plugin_name="chat",
    settings=settings
)

history = ChatHistory()
history.add_user_message("Hi there, who are you?")
history.add_assistant_message("I am Fred. A cute white duck! Miau! 🦆")


input = "I have dementia. What's your name again?"
response = await kernel.invoke(chat_function,user_input=input,chat_history=history)

print(response)
history.add_user_message(input)
history.add_assistant_message(str(response))

No problem at all! 😊 I'm Fred, your friendly white duck! 🦆✨ How can I help you today?


Key concepts to understand about the kernel:

- It's your main entry point for all SK operations
- Manages AI service connections
- Orchestrates function execution
- Handles memory and context

### 2. Plugins


Plugins in Semantic Kernel are collections of related functions.

There are two types of skills/functions:

**Native Plugins**: Traditional Python functions





In [5]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions import KernelArguments


class MathPlugin:
    """Description: MathPlugin provides a set of functions to make Math calculations.

    Usage:
        kernel.add_plugin(MathPlugin(), plugin_name="math")

    Examples:
        {{math.Add}} => Returns the sum of input and amount
        {{math.Multiply}} => Returns the product of input and amount
    """

    @kernel_function(name="Add")
    def add(
        self, input: Annotated[int, "The first number to add"], amount: Annotated[int, "The second number to add"]
    ) -> Annotated[int, "The sum of the two numbers"]:
        """Returns the addition result of the values provided."""
        if isinstance(input, str):
            input = int(input)
        if isinstance(amount, str):
            amount = int(amount)
        return input + amount

    @kernel_function(name="Multiply")
    def multiply(
        self,
        input: Annotated[int, "The first number to multiply"],
        amount: Annotated[int, "The second number to multiply"],
    ) -> Annotated[int, "The product of the two numbers"]:
        """Returns the multiplication result of the values provided."""
        if isinstance(input, str):
            input = int(input)
        if isinstance(amount, str):
            amount = int(amount)
        return input * amount


# Register the plugin with the kernel
math_plugin = kernel.add_plugin(MathPlugin(), "math")

result = await kernel.invoke_prompt(
        "What is 844867564 - 15345674",
        plugin_name="math",
    )
print(result)

The result of \( 844867564 - 15345674 \) is \( 829521890 \).


**Semantic Plugins**: AI-powered functions defined by prompts

In [6]:
pet_name_settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
pet_name_settings.temperature = 0.8

prompt_template = """
Input: {{$input}}
Task: Generate three creative names for a pet {{$animal_type}}.
Requirements:
- Names should be family-friendly
- Each name should be unique
- Include a brief explanation for each name

Output your response in this format:
1. [Name] - [Explanation]
2. [Name] - [Explanation]
3. [Name] - [Explanation]
"""

pet_names = kernel.add_function(
    plugin_name="PetNames",
    function_name="GenerateNames",
    prompt=prompt_template,
    template_format="semantic-kernel",
    prompt_execution_settings=pet_name_settings,
)

result = await kernel.invoke(
        pet_names,
        KernelArguments(input="", animal_type="cat")
    )
print(result)

1. Whisker Biscuit - This playful name combines the delightful imagery of a cat's whiskers with the comforting connotation of a warm biscuit, suggesting a sweet and cuddly personality that's perfect for a loving feline friend.

2. Purrlock Holmes - A clever twist on the famous detective Sherlock Holmes, this name highlights a cat's purring nature while also implying curiosity and intelligence, making it ideal for a cat that loves to investigate its surroundings.

3. Catrina Sparkle - This whimsical name brings to mind a charming cat with a sparkling personality. "Catrina" adds a touch of elegance, while "Sparkle" captures the joyous energy that a playful and affectionate cat brings to a household.


Of course you can combine both types

In [7]:
class InsightPlugin:
    """Description: InsightPlugin provides deep analysis and insights from text.
    It combines emotional analysis, key themes, and actionable takeaways.

    Usage:
        kernel.add_plugin(InsightPlugin(), plugin_name="insight")
    """

    @kernel_function(name="CountWords")
    def count_words(
        self, input: Annotated[str, "The text to count words in"]
    ) -> Annotated[int, "The number of words in the text"]:
        """Returns the number of words in the provided text."""
        return len(input.split())

    @kernel_function(
        name="AnalyzeEmotion", description="Analyzes the emotional undertones and psychological aspects of text"
    )
    async def analyze_emotion(self, input: Annotated[str, "The text to analyze"]) -> str:
        """Analyzes emotional and psychological aspects of the text."""
        prompt = """
        Analyze the emotional and psychological aspects of this text:
        {{$input}}

        Provide insights in this format:
        🎭 Dominant Emotions:
        • [List the main emotions with emoji indicators]

        💭 Psychological Themes:
        • [List key psychological themes]

        🌟 Emotional Intelligence Score (1-10):
        • [Provide a score with explanation]
        """

        function = kernel.add_function(
            plugin_name="EmotionAnalyzer", function_name="Analyze", prompt=prompt, template_format="semantic-kernel"
        )

        return await kernel.invoke(function, KernelArguments(input=input))

    @kernel_function(name="ExtractThemes", description="Identifies key themes and patterns in the text")
    async def extract_themes(self, input: Annotated[str, "The text to analyze"]) -> str:
        """Extracts and analyzes key themes from the text."""
        prompt = """
        Analyze the key themes and patterns in this text:
        {{$input}}

        Present the analysis in this format:
        🎯 Core Themes:
        • [List main themes with brief explanations]

        🔄 Recurring Patterns:
        • [List patterns with examples]

        💡 Hidden Insights:
        • [List subtle or non-obvious insights]
        """

        function = kernel.add_function(
            plugin_name="ThemeAnalyzer", function_name="Analyze", prompt=prompt, template_format="semantic-kernel"
        )

        return await kernel.invoke(function, KernelArguments(input=input))

    @kernel_function(name="GenerateActionItems", description="Creates actionable takeaways from the text")
    async def generate_action_items(self, input: Annotated[str, "The text to analyze"]) -> str:
        """Generates practical action items from the text."""
        prompt = """
        Generate actionable takeaways from this text:
        {{$input}}

        Present them in this format:
        ⚡ Quick Wins:
        • [List immediate actions]

        🎯 Strategic Moves:
        • [List longer-term actions]

        ⚠️ Watch Points:
        • [List potential challenges or considerations]
        """

        function = kernel.add_function(
            plugin_name="ActionGenerator", function_name="Generate", prompt=prompt, template_format="semantic-kernel"
        )

        return await kernel.invoke(function, KernelArguments(input=input))



insight_plugin = kernel.add_plugin(InsightPlugin(), "insight")
sample_text = """
    The startup's journey was a rollercoaster of emotions. The team worked tirelessly for months, 
    facing numerous setbacks but refusing to give up. Their persistence paid off when they finally 
    secured major funding, but this success brought new challenges of scaling quickly while 
    maintaining their innovative culture. The founder often lay awake at night, excited about 
    the possibilities yet anxious about the responsibilities of managing a rapidly growing team.
    """

print("🔍 Deep Text Analysis Demo\n")
print("📝 Analyzing text:", sample_text, "\n")

print("1️⃣ Emotional Analysis:")
emotion_analysis = await kernel.invoke(insight_plugin["AnalyzeEmotion"], KernelArguments(input=sample_text))
print(emotion_analysis, "\n")

print("2️⃣ Theme Analysis:")
theme_analysis = await kernel.invoke(insight_plugin["ExtractThemes"], KernelArguments(input=sample_text))
print(theme_analysis, "\n")

print("3️⃣ Action Items:")
action_items = await kernel.invoke(insight_plugin["GenerateActionItems"], KernelArguments(input=sample_text))
print(action_items)

## Print the amount of words

🔍 Deep Text Analysis Demo

📝 Analyzing text: 
    The startup's journey was a rollercoaster of emotions. The team worked tirelessly for months, 
    facing numerous setbacks but refusing to give up. Their persistence paid off when they finally 
    secured major funding, but this success brought new challenges of scaling quickly while 
    maintaining their innovative culture. The founder often lay awake at night, excited about 
    the possibilities yet anxious about the responsibilities of managing a rapidly growing team.
     

1️⃣ Emotional Analysis:
🎭 Dominant Emotions:
• 💪 Determination - The team's refusal to give up in the face of setbacks demonstrates strong willpower.
• 🎉 Excitement - Securing major funding brings a surge of hope and potential for the future.
• 😟 Anxiety - The founder's worry about managing growth indicates a significant level of stress.
• 🎢 Frustration - The rollercoaster metaphor suggests a mix of ups and downs, highlighting feelings of frustration during s

### 3. Memory

Semantic Kernel provides built-in memory capabilities using embeddings:


In [ ]:
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")


await memory.save_information(
        collection="facts", id="fact1", text=str(emotion_analysis), description="Emotional Analysis"
    )




questions = [
    "What was the result of the emotional analysis",
]

for question in questions:
    print(f"Question: {question}")
    result = await memory.search("facts", question)
    print(f"Answer: {result[0].text}\n")




### 4. Wrapping it up - Emoji Translator

In [ ]:

class EmojiNarratorPlugin:
    """Description: EmojiNarratorPlugin translates stories to and from emoji sequences.

    Usage:
        narrator = EmojiNarratorPlugin(kernel)
    """

    def __init__(self, kernel: Kernel):
        encode_template = """
        Story: {{$story}}
        Style: {{$style}}
        
        Transform this story into a sequence of emojis that:
        1. Captures key plot points
        2. Represents character emotions
        3. Includes relevant symbols for setting
        4. Uses emoji combinations for complex concepts
        5. Maintains narrative flow
        
        Also provide a legend explaining your emoji choices.
        """

        decode_template = """
        Emoji sequence: {{$emoji_sequence}}
        Genre: {{$genre}}
        Tone: {{$tone}}
        
        Create a detailed story from these emojis that:
        1. Interprets each emoji creatively
        2. Builds a coherent narrative
        3. Adds unexpected but logical connections
        4. Includes dialogue and descriptions
        5. Matches the specified genre and tone
        """

        self.encoder = kernel.add_function(
            plugin_name="EmojiNarrator",
            function_name="Encode",
            prompt=encode_template,
            template_format="semantic-kernel"
        )

        self.decoder = kernel.add_function(
            plugin_name="EmojiNarrator",
            function_name="Decode",
            prompt=decode_template,
            template_format="semantic-kernel"
        )

    async def story_to_emoji(self, story: str, style: str = "modern"):
        return await kernel.invoke(
            self.encoder,
            KernelArguments(story=story, style=style)
        )

    async def emoji_to_story(self, emoji_sequence: str, genre: str, tone: str):
        return await kernel.invoke(
            self.decoder,
            KernelArguments(emoji_sequence=emoji_sequence, genre=genre, tone=tone)
        )


narrator = EmojiNarratorPlugin(kernel)

# Encode a story
original_story = "A scientist accidentally creates a time machine but it only works on houseplants"
emoji_version = await narrator.story_to_emoji(original_story, "sci-fi comedy")

# Decode back to a different story
new_story = await narrator.emoji_to_story(
    str(emoji_version),
    genre="philosophical thriller",
    tone="mysterious"
)

print(f"Original: {original_story}\nEmoji: {emoji_version}\nNew Story: {new_story}")
